# Measure a Profile

## Note

All functions in this section using the cluster object as first argument are also internal functions of the cluster object, and should be used as such. They are just explicitely used here for clarity.

Ex:

```
theta, g_t, g_x = compute_tangential_and_cross_components(cl, geometry="flat")
```

should be done by the user as:

```
theta, g_t, g_x = cl.compute_tangential_and_cross_components(geometry="flat")
```

In [ ]:
import matplotlib.pyplot as plt
import clmm
import clmm.polaraveraging
from clmm.polaraveraging import compute_tangential_and_cross_components, make_binned_profile, make_bins
from clmm.plotting import plot_profiles
from clmm.galaxycluster import GalaxyCluster
import clmm.utils as u
import sys
sys.path.append('./support')
import mock_data as mock

Make sure we know which version we're using

In [ ]:
clmm.__version__

### Define cosmology object

In [ ]:
from astropy.cosmology import FlatLambdaCDM
mock_cosmo = FlatLambdaCDM(H0=70., Om0=0.3, Ob0=0.025)

## 1. Generate cluster object from mock data
In this example, the mock data includes: shape noise, galaxies drawn from redshift distribution and photoz errors.

Define toy cluster parameters for mock data generation

In [ ]:
cosmo = mock_cosmo
cluster_id = "Awesome_cluster"
cluster_m = 1.e15
cluster_z = 0.3
concentration = 4
ngals = 1000
Delta = 200

zsrc_min = cluster_z + 0.1 # we only want to draw background galaxies

noisy_data_z = mock.generate_galaxy_catalog(cluster_m,
                                            cluster_z,
                                            concentration,
                                            cosmo,
                                            ngals,
                                            Delta,
                                            'chang13',
                                            zsrc_min = zsrc_min,
                                            shapenoise=0.005,
                                            photoz_sigma_unscaled=0.05)

Loading this into a CLMM cluster object centered on (0,0)

In [ ]:
cluster_ra = 0.0
cluster_dec = 0.0
cl = GalaxyCluster(cluster_id, cluster_ra, cluster_dec, 
                               cluster_z, noisy_data_z)

### 2. Load cluster object containing:
> Lens properties (ra_l, dec_l, z_l)

> Source properties (ra_s, dec_s, e1, e2)
### Note, if loading from mock data, use: 
>> cl = gc.GalaxyCluster.load("GC_from_mock_data.pkl")

In [ ]:
print("Cluster info = ID:", cl.unique_id, "; ra:", cl.ra,
      "; dec:", cl.dec, "; z_l :", cl.z)
print("The number of source galaxies is :", len(cl.galcat))

## 2. Basic checks and plots 
- galaxy positions
- redshift distribution

In [ ]:
f, ax = plt.subplots(1, 2, figsize=(12, 4))

ax[0].scatter(cl.galcat['ra'], cl.galcat['dec'], color='blue', s=1, alpha=0.3)
ax[0].plot(cl.ra, cl.dec, 'ro')
ax[0].set_ylabel('dec', fontsize="large")
ax[0].set_xlabel('ra', fontsize="large")

hist = ax[1].hist(cl.galcat['z'], bins=40)[0]
ax[1].axvline(cl.z, c='r', ls='--')
ax[1].set_xlabel('$z_{source}$', fontsize="large")
xt = {t:f'{t}' for t in ax[1].get_xticks() if t!=0}
xt[cl.z] ='$z_{cl}$'
xto = sorted(list(xt.keys())+[cl.z])
ax[1].set_xticks(xto)
ax[1].set_xticklabels(xt[t] for t in xto)
ax[1].get_xticklabels()[xto.index(cl.z)].set_color('red')
plt.xlim(0, max(xto))
plt.show()

- Check ellipticities

In [ ]:
fig, ax1 = plt.subplots(1, 1)

ax1.scatter(cl.galcat['e1'], cl.galcat['e2'], s=1, alpha=0.2)
ax1.set_xlabel('e1')
ax1.set_ylabel('e2')
ax1.set_aspect('equal', 'datalim')
ax1.axvline(0, linestyle='dotted', color='black')
ax1.axhline(0, linestyle='dotted', color='black')

## 3. Compute and plot shear profiles

### 3.1 Compute angular separation, cross and tangential shear for each source galaxy

- By defaut, `compute_tangential_and_cross_components` uses columns named `e1` and `e2` of the `galcat` table

In [ ]:
theta, e_t, g_x = compute_tangential_and_cross_components(cl, geometry="flat",add_to_cluster=True)
# With the option add_to_cluster the cl object has theta, et and ex new columns 
# (default: takes in columns named 'e1' and 'e2' and save the results in 'et' and 'ex')
cl

- But it's also possible to choose which columns to use for input and output, e.g. Below we're storing the results in `e_tan` and `e_cross` instead (explicitely taking `e1` and `e2` as input)


In [ ]:
theta, e_t, g_x = compute_tangential_and_cross_components(cl, geometry="flat",
                                                      shape_component1='e1', shape_component2='e2', 
                                                      tan_component='e_tan', cross_component='e_cross',
                                                      add_to_cluster=True)
cl

Plot tangential and cross ellipticity distributions for verification, which can be accessed in the galaxy cluster object, cl.

In [ ]:
f, ax = plt.subplots(1, 2, figsize=(12, 4))

ax[0].hist(cl.galcat['et'],bins=50)
ax[0].set_xlabel('$\\epsilon_t$',fontsize='xx-large')

ax[1].hist(cl.galcat['ex'],bins=50)
ax[1].set_xlabel('$\\epsilon_x$',fontsize='xx-large')
ax[1].set_yscale('log')

Compute transversal and cross shear profiles in units defined by user, using defaults binning 

### 3.2 Compute shear profile in radial bins
Given the separations in "radians" computed in the previous step, the user may ask for a binned profile in various projected distance units. The GCData corresponding to the binning profiled is attached as a new attribute of the galaxy cluster object.
#### 3.2.1 Default binning
- default binning using kpc:

In [ ]:
profiles = make_binned_profile(cl, "radians", "kpc", cosmo=cosmo)
cl.profile

Use function to plot the profiles

In [ ]:
fig, ax = plot_profiles(cl,xscale='log')

- default binning using degrees:

In [ ]:
new_profiles = make_binned_profile(cl, "radians", "degrees",cosmo=cosmo)
fig1, ax1 = plot_profiles(cl, "degrees")

#### 3.2.2 User-defined binning 
The users may also provide their own binning, in user-defined units, to compute the transversal and cross shear profiles. The `make_bins` function is provided in `utils.py` and allow for various options. 

- e.g., generate 20 bins between 1 and 6 Mpc, linearly spaced.

In [ ]:
new_bins = make_bins(1, 6, nbins=20, method='evenwidth')

# Make the shear profile in this binning
new_profiles = make_binned_profile(cl, "radians", "Mpc",
                                  bins=new_bins, cosmo=cosmo)

fig1, ax1 = plot_profiles(cl, "Mpc", r_units='Mpc')

- e.g., generate 20 bins between 1 and 6 Mpc, evenly spaced in log space.

In [ ]:
new_bins = make_bins(1, 6, nbins=20, method='evenlog10width')

new_profiles = make_binned_profile(cl, "radians", "Mpc",
                                  bins=new_bins, cosmo=cosmo)
fig1, ax1 = plot_profiles(cl, "Mpc", r_units='Mpc')
ax1.set_xscale('log')

- e.g., generate 20 bins between 1 and 6 Mpc, each contaning the same number of galaxies

In [ ]:
# First, convert the source separation table to Mpc
seps = u.convert_units(cl.galcat["theta"], "radians", "Mpc", redshift=cl.z, cosmo=cosmo)

new_bins = make_bins(1, 6, nbins=20, method='equaloccupation', source_seps=seps)
new_profiles = make_binned_profile(cl, "radians", "Mpc",bins=new_bins, cosmo=cosmo)

print(f"number of galaxies in each bin: {list(cl.profile['n_src'])}")
fig1, ax1 = plot_profiles(cl, "Mpc", r_units='Mpc')

#### 3.2.3 Other individual profile quantities may also be accessed 

In [ ]:
plt.title('Average redshift in radial bins')
plt.errorbar(new_profiles['radius'], new_profiles['z'],
             new_profiles['z_err'], marker = 'o')
plt.axhline(cl.z, linestyle='dotted', color='r')
plt.text(1, cl.z*1.1, '$z_{cl}$', color='r')
plt.xlabel("Radius [Mpc]")
plt.ylabel('$\langle z\\rangle$')
plt.show()

## 4. Focus on some options
### 4.1. `gal_ids_in_bins` option 
adds a `gal_id` field to the profile GCData. For each bin of the profile, this is filled with the list of galaxy IDs for the galaxies that have fallen in that bin.

In [ ]:
profiles = make_binned_profile(cl, "radians", "Mpc", cosmo=cosmo, gal_ids_in_bins=True)

In [ ]:
# Here the list of galaxy IDs that are in the first bin of the tangential shear profile
import numpy as np
gal_list = profiles['gal_id'][0]
print(gal_list)

### 4.2. User-defined naming scheme
The user may specify which columns to use from the `galcat` table to perform the binned average. If none is specified, the code looks for columns names `et` and `ex`. Below, we average in bins the columns`e_tan` and `e_cross` of `galcat` and store the results in the columns `g_tan` and `g_cross` of the `profile` table of the cluster object.

In [ ]:
cl.make_binned_profile("radians", "kpc", cosmo=cosmo, 
                       tan_component_in='e_tan', cross_component_in='e_cross',
                       tan_component_out='g_tan', cross_component_out='g_cross');
cl.profile

The user may also define the name of the output table attribute. Below, we asked the binned profile to be saved into the `reduced_shear_profile` attribute

In [ ]:
cl.make_binned_profile("radians", "kpc", cosmo=cosmo, 
                       tan_component_in='e_tan', cross_component_in='e_cross',
                       tan_component_out='g_tan', cross_component_out='g_cross',
                       table_name='reduced_shear_profile');
cl.reduced_shear_profile

### 4.3 Compute a DeltaSigma profile instead of a shear profile

The `is_deltasigma` option allows the user to return a cross and tangential $\Delta\Sigma$ (excess surface density) value for each galaxy in the catalog, provided `galcat` contains the redshifts of the galaxies and provided a cosmology is passed to the function. The columns `DeltaSigma_tan` and `DeltaSigma_cross` are added to the `galcat` table.

In [ ]:
theta, DS_t, DS_x = compute_tangential_and_cross_components(cl, geometry="flat",
                                                      shape_component1='e1', shape_component2='e2', 
                                                      tan_component='DeltaSigma_tan', cross_component='DeltaSigma_cross',
                                                      add_to_cluster=True, cosmo=cosmo, is_deltasigma = True)
cl

The binned profile is obtained, as before. Below, we use the values obtained from the previous step to compute the binned profile. The latter is saved in a new `DeltaSigma_profile` table of the GalaxyCluster object.

In [ ]:
cl.make_binned_profile("radians", "Mpc", cosmo=cosmo, 
                       tan_component_in='DeltaSigma_tan', cross_component_in='DeltaSigma_cross',
                       tan_component_out='DeltaSigma_tan', cross_component_out='DeltaSigma_cross',
                       table_name='DeltaSigma_profile');

In [ ]:
cl.DeltaSigma_profile

In [ ]:
plt.errorbar(cl.DeltaSigma_profile['radius'], cl.DeltaSigma_profile['DeltaSigma_tan'],
             cl.DeltaSigma_profile['DeltaSigma_tan_err'], marker = 'o')
plt.title('DeltaSigma profile')
plt.xlabel("Radius [Mpc]")
plt.ylabel('$\Delta\Sigma [h\; M_\odot\; pc^{-2}]$')
plt.show()